In [8]:
import pandas as pd
from pandasql import sqldf
import numpy as np

In [12]:
# В таблице user_actions сохраняются действия покупателей в интернет магазине, структура таблицы

# user_id    - id пользователи
# product_id - id товара
# action     - действие, просмотр товара, сохранение в корзину, покупка
# time       - время совершения события
# date       - дата события

# Для каждого дня расчитайте, какой процент просмотров товаров завершился покупкой

In [13]:
# Создаем список id пользователей и товара
all_user_ids = np.arange(1, 10000)
all_product_ids = np.arange(1, 100)

In [20]:
# Создаем выборку из 10000 id пользователей и товаров
n = 10000
user_ids    = np.random.choice(all_user_ids, n)
product_ids = np.random.choice(all_product_ids, n)

In [21]:
# Создаем список с датами с шагом в одну минуту начиная от 2022-01-01
start_date = pd.to_datetime('2022-01-01')

times = pd.date_range(start_date, periods=n, freq='1min')

In [25]:
# Создаем ДатаФрейм и добавлем столбец с action
user_actions = pd.DataFrame ({'user_id' : user_ids,
                              'product_id' : product_ids,
                              'time' : times})

In [26]:
user_actions['action'] = 'view'

In [27]:
#
def generate_funel_actions(user_id, product_id, time):
    to_cart = 0.2
    to_purchase = 0.4
    
    df = pd.DataFrame()
    
    if np.random.binomial(1, to_cart, 1) [0]:
        df = pd.DataFrame({
                            'user_id'    : user_id,
                            'product_id' : product_id,
                            'time'       : time + pd.Timedelta(5, unit = 's'),
                            'action'     : 'add to cart'}, index = [0])
        if np.random.binomial(1, to_purchase, 1) [0]:
            df_purchase = pd.DataFrame({
                                        'user_id'    : user_id,
                                        'product_id' : product_id,
                                        'time'       : time + pd.Timedelta(5, unit = 's'),
                                        'action'     : 'purchase'}, index = [0])
            df = df.append(df_purchase)
    return df

In [28]:
to_cart_df = pd.DataFrame()

In [29]:
for index, row in user_actions.iterrows():
    user_df = generate_funel_actions(row['user_id'], row['product_id'], row['time'])
    to_cart_df = to_cart_df.append(user_df)

In [31]:
user_actions = user_actions.append(to_cart_df)
user_actions = user_actions.sort_values('time')

In [32]:
user_actions['date'] = user_actions.time.dt.date

In [33]:
user_actions.head()

,user_id,product_id,time,action,date
0,773,83,2022-01-01 00:00:00,view,2022-01-01
0,773,83,2022-01-01 00:00:05,add to cart,2022-01-01
1,6594,66,2022-01-01 00:01:00,view,2022-01-01
0,6594,66,2022-01-01 00:01:05,add to cart,2022-01-01
0,6594,66,2022-01-01 00:01:05,purchase,2022-01-01


In [40]:
q = """SELECT date,
              views,  
              carts,
              purchases,
              100 * purchases / views as purchases_percantage
        FROM(
            SELECT date,
                count(case when action = 'view'         then 1 else NULL end) as views,
                count(case when action = 'add to cart' then 1 else NULL end) as carts,
                count(case when action = 'purchase'   then 1 else NULL end) as purchases
            FROM user_actions
            GROUP BY date);"""

In [41]:
sqldf(q)

,date,views,carts,purchases,purchases_percantage
0,2022-01-01,1440,283,109,7
1,2022-01-02,1440,311,130,9
2,2022-01-03,1440,277,108,7
3,2022-01-04,1440,303,116,8
4,2022-01-05,1440,281,114,7
5,2022-01-06,1440,291,118,8
6,2022-01-07,1360,285,112,8
